# **Cassava leaf disease classification: ResNext Model with Test Time Augmentation!**

*We started by importing all the necessary libraris needed for this implementation, and we'll try as much as we can to comment every line of code!*

In [ ]:
!pip install ttach # Intalling the test time augmentation package

In [4]:
# We started by importing all the necessary libraris needed for this implementation.
import os # functionalities for interacting with the operating system (reading files etc..).
import sys # import system modules
import glob # search for files that match specific patterns
import torch # pytorch package, which contains other utilities for deep learning.
import torchvision
import pandas as pd
from tqdm import tqdm
from torchvision.models import alexnet

import numpy    as np
import datetime as dt
import torch.nn as nn
import ttach as tta # augmenting test time data and averaging the predictions

import torch.nn.functional as F
import matplotlib.pyplot   as plt

from PIL               import Image
from torch.utils.data  import Dataset
from torch.autograd    import Variable
from torch.optim       import lr_scheduler

from torch.utils.data  import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision       import transforms, datasets, models
from os                import listdir, makedirs, getcwd, remove
from os.path           import isfile, join, abspath, exists, isdir, expanduser
import gc
# torch.cuda.empty_cache()
gc.collect()
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"


import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

### **Loading the dataset**

In [9]:
data_path = "/kaggle/input/ammi-2024-computer-vision/"
train_path = join(data_path, "train/train")
test_path = join(data_path,"test/test")
extraimage_path = join(data_path, "extraimages/extraimages")
print("Successfully loaded the datset!")

### **Device configuration**

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### **Transforming the dataset**

*Transformin the dataset is essential for augmentation.*

In [10]:
# Transformations for both the training and testing data
mean=[0.485, 0.456, 0.406] # mean and standard deviation for RGB channels.
std=[0.229, 0.224, 0.225]

# randomly crop all the training images, convert them to a tesnor and normalize them.
train_transforms  = transforms.Compose([
    transforms.Resize((580,580)),
    #transforms.CenterCrop(448),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees = 30),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    
])


test_transforms = transforms.Compose([ transforms.Resize((580,580)),
                                       #transforms.Resize(255),
                                       #transforms.CenterCrop(448),
                                       transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])



### **Implementing the datasset class**

*Easier to access and handle images during training*

In [11]:
class CassavaDataset(Dataset):
    def __init__(self, path, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
            
        return im.view(3, 580, 580), classCategory

In [12]:
train_data = CassavaDataset(train_path, transform=train_transforms)
test_data = CassavaDataset(test_path, transform=test_transforms)

### **Creating data indices for training and validation**

In [13]:
validation_split = 0.1
shuffle_dataset = True
random_seed= 42

dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

### **Preparing the data samplers and loaders**

In [14]:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=8,
                                             sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=8,
                                             sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=8)

In [16]:
torch.manual_seed(random_seed)  # Fix the random seed

model = torchvision.models.resnext50_32x4d(pretrained=True) # Loading the resnext50_32x4d pretrained model.

# Set the final linear layer to the number of classes.
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)
model = model.to(device)

criterion = nn.CrossEntropyLoss()

hypergrad_lr = 1e-9 # hypergradient learning rate.
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=3e-3)
hypergrad_optimizer = torch.optim.Adam(model.parameters(), lr=hypergrad_lr)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 144MB/s] 


In [19]:
# Define the path to the folder containing the subfolders
path = train_path

# Get the list of subfolders
subfolders = [f.path for f in os.scandir(path) if f.is_dir()]

# Create a list to store the number of images in each subfolder
num_images = []

# Iterate over the subfolders and count the number of images
for subfolder in subfolders:
    num_images.append(len(os.listdir(subfolder)))

# Compute class weights
class_counts = num_images  # Number of samples per class
total_samples = sum(class_counts)
class_weights = [total_samples / (len(class_counts) * count) for count in class_counts]
class_weights = torch.FloatTensor(class_weights).to(device)

In [20]:
num_classes = 5
num_epochs = 10
learning_rate = 1e-4
hypergrad_lr = 1e-9

model1 = torchvision.models.resnext101_64x4d(pretrained=True) # Loading the resnext101_64x4d pretrained model
model1.avgpool = nn.AdaptiveAvgPool2d((1, 1))
num_features = model1.fc.in_features
model1.fc = nn.Linear(num_features, num_classes)

criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = torch.optim.Adam(model1.parameters(), lr=learning_rate)
hypergrad_optimizer = torch.optim.Adam(model1.parameters(), lr=hypergrad_lr)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_64X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_64X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext101_64x4d-173b62eb.pth" to /root/.cache/torch/hub/checkpoints/resnext101_64x4d-173b62eb.pth
100%|██████████| 319M/319M [00:04<00:00, 76.0MB/s] 


### **The train function**

In [ ]:
def train(model, criterion, data_loader, valid_loader, optimizer, num_epochs):
    # Move model to the device (CPU or GPU).
    model = model.to(device)
    
    # Exponential moving average of the loss.
    ema_loss = None
    best_acc = 0

    # Lists to store losses and accuracies.
    losses = []
    training_accuracies = []
    validation_accuracies = []

    print('----- Training Loop -----')
    
    # Loop over epochs.
    for epoch in range(num_epochs):
        # Make sure model is in training mode.
        model.train()
        correct = 0
        
        # Loop over data.
        for batch_idx, (features, target) in enumerate(data_loader):
            torch.cuda.empty_cache()
            
            # Forward pass.
            output = model(features.to(device))
            loss = criterion(output.to(device), target.to(device))
            
            # Backward pass.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Compute the training accuracy
            with torch.no_grad():
                pred = output.argmax(dim=1, keepdim=True)
                
                # Count number of correct predictions.
                correct += pred.cpu().eq(target.view_as(pred)).sum().item()
            
            # Exponential moving average of the loss.
            if ema_loss is None:
                ema_loss = loss.item()
            else:
                ema_loss += (loss.item() - ema_loss) * 0.01
        
        # Compute the training accuracy.
        train_score = 100. * correct / len(data_loader.sampler)
        
        # Compute the validation accuracy.
        valid_score = test(model, valid_loader)
        
        # Store the losses and accuracies.
        losses.append(ema_loss)
        training_accuracies.append(train_score)
        validation_accuracies.append(valid_score)
        
        # Save the best model.
        if valid_score > best_acc:
            best_acc = valid_score
            torch.save(model, 'get_resnext101_weighted.pth')
        
        # Print out progress at the end of epoch.
        print(f'Epoch: {epoch} \tLoss: {ema_loss:.12f} \tTraining Accuracy: {train_score:.12f} \tValidation Accuracy: {valid_score:.12f}')

    return losses, training_accuracies, validation_accuracies


### **The test function**

In [21]:
def test(model, data_loader):
    # Make sure the model is in evaluation mode.
    model.eval()
    correct = 0
    print('----- Model Evaluation -----')
    
    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():
        # Loop over test data.
        for features, target in data_loader:
            # Forward pass.
            output = model(features.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            
            # Count number of correct predictions.
            correct += pred.cpu().eq(target.view_as(pred)).sum().item()
    
    # Print test accuracy.
    percent = 100. * correct / len(valid_sampler)
    print(f'Test accuracy: {correct} / {len(valid_sampler)} ({percent:.0f}%)')
    
    # torch.save(model.state_dict(), 'model.ckpt')
    return percent


### **Start training**

In [18]:
num_epochs = 10 # set number of epochs for training
losses, training_accuracies, validation_accuracies = train(model1, criterion, train_loader, valid_loader, optimizer, num_epochs)

----- Training Loop -----
----- Model Evaluation -----
Test accuracy: 473 / 565 (84%)
Epoch: 0 	Loss: 0.578116055279 	Training Accuracy: 77.843252799057 	 Validation Accuracy: 83.716814159292
----- Model Evaluation -----
Test accuracy: 510 / 565 (90%)
Epoch: 1 	Loss: 0.446727400861 	Training Accuracy: 85.955607935573 	 Validation Accuracy: 90.265486725664
----- Model Evaluation -----
Test accuracy: 512 / 565 (91%)
Epoch: 2 	Loss: 0.344238200770 	Training Accuracy: 87.762718522884 	 Validation Accuracy: 90.619469026549
----- Model Evaluation -----
Test accuracy: 508 / 565 (90%)
Epoch: 3 	Loss: 0.310996923967 	Training Accuracy: 89.668041642114 	 Validation Accuracy: 89.911504424779
----- Model Evaluation -----
Test accuracy: 505 / 565 (89%)
Epoch: 4 	Loss: 0.299683268496 	Training Accuracy: 91.376939697505 	 Validation Accuracy: 89.380530973451
----- Model Evaluation -----
Test accuracy: 509 / 565 (90%)
Epoch: 5 	Loss: 0.243495238712 	Training Accuracy: 92.614417599686 	 Validation Accu

In [19]:
# validating the performance of model1
test(model1, valid_loader)

----- Model Evaluation -----
Test accuracy: 498 / 565 (88%)


88.14159292035399

### **Test Time Augmentation with Differenet Transformations**

In [20]:
tta_model = tta.ClassificationTTAWrapper(model1, tta.aliases.five_crop_transform(500,500))

In [21]:
test(tta_model, valid_loader) # validating the performance of the model with tta

----- Model Evaluation -----
Test accuracy: 499 / 565 (88%)


88.31858407079646

In [22]:
tta_model2 = tta.ClassificationTTAWrapper(model1, tta.aliases.flip_transform()) # test time augmentation with flip transformation


In [23]:
test(tta_model2, valid_loader) # Validating the performance of model2 with tta

----- Model Evaluation -----
Test accuracy: 502 / 565 (89%)


88.84955752212389

In [7]:
loaded_model = torch.load('/kaggle/input/modelweight/get_resnext101_weighted.pth') # Loading the model weights

In [25]:
test(loaded_model, valid_loader) #Validating performance using the model's weights

----- Model Evaluation -----
Test accuracy: 509 / 565 (90%)


90.08849557522124

In [26]:
tta_model = tta.ClassificationTTAWrapper(loaded_model, tta.aliases.five_crop_transform(484,484))
test(tta_model, valid_loader)

----- Model Evaluation -----
Test accuracy: 513 / 565 (91%)


90.79646017699115

In [27]:
tta_model2 = tta.ClassificationTTAWrapper(loaded_model, tta.aliases.flip_transform()) #Second TTA Model with Flip Transform
test(tta_model2, valid_loader)


----- Model Evaluation -----
Test accuracy: 514 / 565 (91%)


90.97345132743362

In [28]:
tta_model = tta.ClassificationTTAWrapper(loaded_model, tta.aliases.five_crop_transform(500,500)) #Third TTA Model with Flip Transform
tta_model3 = tta.ClassificationTTAWrapper(tta_model, tta.aliases.flip_transform())
test(tta_model3, valid_loader)

----- Model Evaluation -----
Test accuracy: 511 / 565 (90%)


90.4424778761062

to run

In [29]:
tta_model4 = tta.ClassificationTTAWrapper(loaded_model, tta.aliases.ten_crop_transform(500,500)) # Fourth TTA Model with Flip Transform
test(tta_model4, valid_loader) 

----- Model Evaluation -----
Test accuracy: 512 / 565 (91%)


90.61946902654867

In [30]:
tta_model = tta.ClassificationTTAWrapper(loaded_model, tta.aliases.five_crop_transform(500,500)) #Fifth TTA Model with Flip Transform
tta_model5 = tta.ClassificationTTAWrapper(tta_model, tta.aliases.ten_crop_transform(500,500))
tta_model6 = tta.ClassificationTTAWrapper(tta_model5, tta.aliases.flip_transform())
test(tta_model6, valid_loader)

----- Model Evaluation -----
Test accuracy: 514 / 565 (91%)


90.97345132743362

### **Predict function**

In [34]:
def predict(model, loader):
    model.eval()
    test_dataloader =  loader
    preds = []
    with torch.no_grad():
        for test_images, test_labels in tqdm(test_dataloader):
            test_images = test_images.to(device)
            test_labels = test_labels.to(device)

            output = model(test_images)

            _, predicted = torch.max(output, 1)
            preds.extend(predicted.cpu().data.numpy())


    return preds

In [ ]:
preds6 = predict(tta_model2, test_loader) # making prediction using model 2 with tta

100%|██████████| 472/472 [1:20:22<00:00, 10.22s/it]


In [ ]:
preds7 = predict(tta_model3, test_loader) # making prediction using model 3 with tta

100%|██████████| 472/472 [22:24<00:00,  2.85s/it]


In [ ]:
name = [test_data.file_list[i][-1].split('/')[-1] for i in range(len(test_data.file_list))]

### **Preparing the submission file**

In [ ]:
sample = pd.read_csv('/kaggle/input/ammi-2024-computer-vision/sample_submission_file.csv')
sample['Id'] = name

# Mapping from class indices to class names
mapping = {0: 'cmd', 1: 'cbb', 2: 'cbsd', 3: 'healthy', 4: 'cgm'}

# Convert predictions to class names
new_preds = [mapping[pred] for pred in preds7]
sample['Category'] = new_preds

# Save the submission file
sample.to_csv('submission.csv', index=False)
sample.head()
print('submission file created successfully!')

,Category,Id
0,cgm,test-img-1448.jpg
1,cmd,test-img-768.jpg
2,cmd,test-img-3481.jpg
3,cmd,test-img-1475.jpg
4,cgm,test-img-2498.jpg
